<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp drive/'My Drive'/jigsaw-toxic-comment-classification-challenge.zip .

In [44]:
!unzip jigsaw-toxic-comment-classification-challenge.zip
!mkdir tmp_lm/

Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace test_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_labels.csv         
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [0]:
arr = df.columns

In [48]:
classes = arr[2:]
classes = np.asarray(classes)
classes = classes.tolist()
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [0]:
def make_data(df):
  data=[]
  lab=[]
  for row,index in df.iterrows():
    temp=[]
    for class_name in classes:
      if index[class_name]==1:
        temp.append(class_name)
      if len(temp)==0: 
        temp.append("no_class")
    temp = ' '.join(temp)
    lab.append(temp)
    data.append(index["comment_text"])
  X = pd.DataFrame({"labels":lab,"text":data})
  return X

In [0]:
X = make_data(df)
X.to_csv("clean_train.csv",index=False)
#X = make_data(df_test)
#X.to_csv("clean_test.csv",index=False)


In [0]:
from fastai.text import *
path = '.'

In [0]:
data_lm = TextDataBunch.from_csv(path, 'clean_train.csv', test="test.csv", text_cols=1,max_vocab=60000,min_freq=1, label_cols=0,label_delim=" ")

In [64]:
data_lm.show_batch()

text,target
xxbos xxmaj take that ! \n \n xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in,obscene;severe_toxic;toxic
"xxbos "" ( ) you are xxup such a fucking twat flap \n \n xxmaj position / / xxmaj artist , xxmaj band , or xxmaj group / / xxmaj song xxmaj title / / xxmaj weeks xxmaj spent xxmaj at # 1 / / xxmaj year \n \n 1 . xxmaj mariah xxmaj carey & xxmaj boyz xxup ii xxmaj men- "" "" xxmaj one xxmaj sweet",toxic
xxbos ' ' ' jforget xxup fuck xxup you xxup faggot xxup gay xxup ass xxup chink xxup nigger xxup cocksucking xxup piece xxup of xxup shit jforget xxup fuck xxup you xxup faggot xxup gay xxup ass xxup chink xxup nigger xxup cocksucking xxup piece xxup of xxup shit jforget xxup fuck xxup you xxup faggot xxup gay xxup ass xxup chink xxup nigger xxup cocksucking xxup piece xxup,identity_hate;insult;obscene;severe_toxic;toxic
"xxbos "" \n \n xxmaj hi , thanks for your thoughts . i have to disagree about your xxunk - argument . xxmaj you say "" "" xxmaj germany "" "" can refer to either "" "" xxmaj west xxmaj germany "" "" or "" "" xxmaj east xxmaj germany "" "" . xxmaj this is - as i already stated - simply not correct , since there never",no_class
"xxbos xxmaj merge of xxmaj county articles \n \n i will be proposing that all the xxmaj county articles in xxmaj england be merged into several articles , representing the 9 official regions of xxmaj england . xxmaj this involves the following xxunk \n \n xxmaj cleveland xxmaj scout xxmaj county ( xxmaj the xxmaj scout xxunk xxmaj scout xxmaj county ( xxmaj the xxmaj scout xxmaj association",no_class


In [69]:
data_lm.save()
!mv data_save.pkl tmp_lm/
data = load_data("tmp_lm/",bs=64)
data.show_batch()

text,target
xxbos xxmaj take that ! \n \n xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in xxup the xxup ass xxup in,obscene;severe_toxic;toxic
"xxbos i xxup can xxup still xxup post xxup with xxup this xxup computer ... i xxup said xxup block xxup me , xxup come xxup the xxup fuck xxup down xxup here xxup and xxup arrest xxup me ... xxup san xxup diego xxup california , xxup chula xxup vista , xxup fucking xxup get xxup your xxup information xxup right xxup faggot shithead!!i xxup can xxup still xxup",insult;obscene;severe_toxic;toxic
"xxbos xxmaj sam xxmaj guest \n xxmaj instructor : xxmaj todd , j \n xxmaj political xxmaj science 202 \n 11 / 3 / 10 \n xxmaj impacts the 2010 xxmaj midterm xxmaj congressional xxmaj elections has on the \n xxmaj presidential xxmaj election in 2012 \n tabin the 2010 midterm congressional elections , the xxmaj republican party took control of the xxmaj house",no_class
"xxbos "" \n \n xxmaj hi . xxmaj the issue here is very simple . xxmaj the article currently states there is "" "" xxup no "" "" evidence that xxup ect causes brain damage in humans or animals . xxmaj the article also states that there are "" "" xxup no "" "" recent reviews that found such evidence . xxmaj both of these claims in the article",no_class
"xxbos "" \n \n xxmaj from xxmaj norway ; xxmaj denmark ; xxmaj iceland ; xxmaj scotland ; xxunk etc . ) ; xxmaj wales ; xxmaj ireland ; xxmaj basques & xxmaj xxunk data ) \n \n by xxmaj gunnar xxmaj thompson \n \n "" "" xxmaj on xxunk 's map , the northwestern continent is called "" "" xxmaj xxunk . "" "" xxmaj this",no_class


In [0]:
### dont run language model. out of memory error

In [0]:
learn = language_model_learner(data, arch=AWD_LSTM, drop_mult=0.3)


In [0]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))


In [0]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [0]:
learn.save_encoder('fine_tuned_enc')


In [0]:
#run from here

In [0]:
learn = text_classifier_learner(data, drop_mult=0.5, arch=AWD_LSTM, metrics=accuracy_thresh)
#learn.load_encoder('fine_tuned_enc')
learn.freeze()

In [71]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [89]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy_thresh,time
0,0.159221,0.156371,0.954070,07:14


In [0]:
learn.save('first')
learn.load('first');

In [91]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy_thresh,time
0,0.131026,0.129571,0.958600,08:55


In [0]:
learn.save('second')
learn.load('second');

In [93]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy_thresh,time
0,0.087120,0.084733,0.971948,10:33


In [0]:
learn.save('third')
learn.load('third');

In [95]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy_thresh,time
0,0.079280,0.074465,0.975045,17:12
1,0.071430,0.068705,0.977055,16:37


In [0]:
learn.save('fourth')

In [155]:
len(data.test_ds.x)

153164

In [0]:
p = learn.get_preds(ds_type=DatasetType.Test)

In [177]:
p[0]

tensor([[9.3498e-04, 3.4842e-04, 9.9041e-01,  ..., 7.1059e-04, 1.1101e-03,
         9.5192e-03],
        [2.5991e-01, 9.2987e-01, 3.1274e-03,  ..., 6.7471e-01, 2.8228e-01,
         9.9683e-01],
        [1.7404e-02, 1.2004e-01, 4.8751e-01,  ..., 1.7136e-02, 2.2684e-02,
         5.1134e-01],
        ...,
        [1.6373e-02, 8.4300e-03, 6.6751e-01,  ..., 5.6462e-02, 5.1303e-02,
         3.3124e-01],
        [3.8133e-03, 9.5615e-04, 9.3866e-01,  ..., 1.0171e-02, 1.7269e-02,
         6.0951e-02],
        [4.0446e-03, 1.1154e-03, 9.3735e-01,  ..., 1.0251e-02, 1.7570e-02,
         6.2275e-02]])

In [131]:
learn.data.classes

['identity_hate',
 'insult',
 'no_class',
 'obscene',
 'severe_toxic',
 'threat',
 'toxic']

In [0]:
class_rank = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
class_in_pred = []
df_test['toxic'] = Series(np.random.randn(sLength), index=df_test.index)
df_test['severe_toxic'] = Series(np.random.randn(sLength), index=df_test.index)
df_test['obscene'] = Series(np.random.randn(sLength), index=df_test.index)
df_test['threat'] = Series(np.random.randn(sLength), index=df_test.index)
df_test['insult'] = Series(np.random.randn(sLength), index=df_test.index)
df_test['identity_hate'] = Series(np.random.randn(sLength), index=df_test.index)

In [185]:
pred = np.asarray(p[0])
pred.shape

(153164, 7)

In [184]:
len(df_test['id'])

153164

In [0]:
df_test['identity_hate'] = pred[:,0]
df_test['insult'] = pred[:,1]
df_test['obscene'] = pred[:,3]
df_test['severe_toxic'] = pred[:,4]
df_test['threat'] = pred[:,5]
df_test['toxic'] = pred[:,6]



In [204]:
df_temp_test = df_test
df_temp_test= df_temp_test.drop(columns=['comment_text'])
df_temp_test

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.009519,0.000711,0.000539,1.110102e-03,0.000348,0.000935
1,0000247867823ef7,0.996834,0.674710,0.957339,2.822792e-01,0.929873,0.259908
2,00013b17ad220c46,0.511339,0.017136,0.098788,2.268369e-02,0.120037,0.017404
3,00017563c3f7919a,0.974925,0.295061,0.873362,7.579489e-02,0.791521,0.145640
4,00017695ad8997eb,0.690902,0.024076,0.164201,2.596170e-02,0.180081,0.020175
5,0001ea8717f6de06,0.432565,0.008754,0.074600,1.076530e-02,0.091066,0.013630
6,00024115d4cbde0f,0.999411,0.772406,0.992639,2.262292e-01,0.979823,0.385760
7,000247e83dcc1211,0.666831,0.023306,0.162338,2.250535e-02,0.184695,0.023148
8,00025358d4737918,0.997865,0.674117,0.977991,2.021479e-01,0.952262,0.289713
9,00026d1092fe71cc,0.758579,0.031912,0.186140,3.686348e-02,0.228355,0.025075


In [0]:
df_temp_test.to_csv("submission.csv",index=False)


In [196]:
!ls

clean_train.csv					   sample_submission.csv
download-all					   submission.csv
drive						   test.csv
jigsaw-toxic-comment-classification-challenge.zip  test_labels.csv
models						   tmp_lm
sample_data					   train.csv


In [0]:
from google.colab import files
files.download('submission.csv')